In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd

/Users/haripat/Desktop/SF/protein/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_excel("/Users/haripat/Desktop/SF/protein/data/protein_constructs_w_label_masks.xlsx")[['rcsb_id', 'rcsb_entity_ids', 'uniprot_seq','pbd_id', 'pdb_sequence_sanitized', 'label_mask']]
df.head()

,rcsb_id,rcsb_entity_ids,uniprot_seq,pbd_id,pdb_sequence_sanitized,label_mask
0,2GD8,1,MSHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKP...,SHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPL...,SHHWGYGKHNGPEHWHKDFPIAKGERQSPVDIDTHTAKYDPSLKPL...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2GDD,1,MLNLLLLALPVLASRAYAAPAPGQALQRVGIVGGQEAPRSKWPWQV...,IVGGQEAPRSKWPWQVSLRVHGPYWMHFCGGSLIHPQWVLTAAHCV...,IVGGQEAPRSKWPWQVSLRVHGPYWMHFCGGSLIHPQWVLTAAHCV...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2GDE,1,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,TFGSGEADCGLRPLFEKKSLEDKTERELLESYIDGR,TFGSGEADCGLRPLFEKKSLEDKTERELLESYIDGR,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,2GDE,2,MAHVRGLQLPGCLALAALCSLVHSQHVFLAPQQARSLLQRVRRANT...,IVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLL...,IVEGSDAEIGMSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLL...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,2GDO,1,MAVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAV...,MAVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAV...,MAVPFVEDWDLVQTLGEGAYGEVQLAVNRVTEEAVAVKIVDMKRAV...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [3]:
model_name = "facebook/esm2_t33_650M_UR50D"

tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False)
model = AutoModel.from_pretrained(model_name)

model.eval()

Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


EsmModel(
  (embeddings): EsmEmbeddings(
    (word_embeddings): Embedding(33, 1280, padding_idx=1)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): EsmEncoder(
    (layer): ModuleList(
      (0-32): 33 x EsmLayer(
        (attention): EsmAttention(
          (self): EsmSelfAttention(
            (query): Linear(in_features=1280, out_features=1280, bias=True)
            (key): Linear(in_features=1280, out_features=1280, bias=True)
            (value): Linear(in_features=1280, out_features=1280, bias=True)
            (rotary_embeddings): RotaryEmbedding()
          )
          (output): EsmSelfOutput(
            (dense): Linear(in_features=1280, out_features=1280, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (LayerNorm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        )
        (intermediate): EsmIntermediate(
          (dense): Linear(in_features=1280, out_features=5120, bias=True)
        )
        (output): EsmOut

# Exploration

In [4]:
sequence = df.loc[0]['uniprot_seq']
tokens = tokenizer(sequence, return_tensors="pt")
print(tokens["input_ids"].shape)

torch.Size([1, 262])


In [5]:
with torch.no_grad():
    outputs = model(**tokens)
    embeddings = outputs.last_hidden_state
    print(embeddings.shape)

torch.Size([1, 262, 1280])


In [7]:
tokens = tokenizer(
    list(df['uniprot_seq'])[0:10],  
    return_tensors="pt",       
    padding=True,              
    truncation=False            
)

In [8]:
print(tokens["input_ids"].shape)

torch.Size([10, 2479])


In [9]:
with torch.no_grad():
    outputs = model(**tokens)
    embeddings = outputs.last_hidden_state

print(embeddings.shape)

torch.Size([10, 2479, 1280])


In [10]:
mask = tokens['attention_mask'].unsqueeze(-1)      
summed = torch.sum(embeddings * mask, dim=1)       
counts = torch.sum(mask, dim=1)                    
seq_embeddings = summed / counts                   

In [11]:
seq_embeddings.shape

torch.Size([10, 1280])

In [23]:
len(list(df['uniprot_seq'])[10])

198

In [12]:
embeddings[0]

tensor([[ 0.0778, -0.0644,  0.0537,  ..., -0.3008,  0.1563, -0.0535],
        [ 0.0156,  0.0482, -0.0428,  ..., -0.0294, -0.0963,  0.1326],
        [-0.0059,  0.1675, -0.0167,  ..., -0.0887,  0.0805, -0.2506],
        ...,
        [ 0.2190, -0.3089, -0.0842,  ..., -0.3312,  0.0624, -0.3568],
        [ 0.0688, -0.0475, -0.3433,  ..., -0.3868, -0.1393, -0.4313],
        [ 0.2147, -0.2791, -0.2702,  ..., -0.1834, -0.1744, -0.3504]])

# Processing